# Modelling Guide for Predictive Maintenance

This notebook outlines the process of implementing a predictive maintenance model using a sample scenario where machine failures occur due to specific component malfunctions. The objective is to predict these failures. The example datasets illustrate key steps in predictive maintenance, including feature engineering, labeling, training, and evaluation.


## Example
The example in this notebook considers a system with multiple components and sensors.

![](img/machine.png)



## Outline

- [Problem Description](#Problem-Description)
- [Data Sources](#Data-Sources)
   - [Telemetry](#Telemetry)
   - [Errors](#Errors)
   - [Maintenance](#Maintenance)
   - [Machines](#Machines)
   - [Failures](#Failures)
- [Feature Engineering](#Feature-Engineering)
  - [Lag Features from Telemetry](#Lag-Features-from-Telemetry)
  - [Lag Features from Errors](#Lag-Features-from-Errors)
  - [Days Since Last Replacement from Maintenance](#Days-Since-Last-Replacement-from-Maintenance)
  - [Machine Features](#Machine-Features)
- [Label Construction](#Label-Construction)
- [Modelling](#Modelling)
  - [Training, Validation and Testing](#Training,-Validation-and-Testing)
  - [Evaluation](#Evaluation)
- [Summary](#Summary)

## Problem Description
Businesses in asset-heavy industries, like manufacturing, face substantial costs due to production delays caused by mechanical problems. To mitigate the costly impact of downtime, these businesses seek to predict such issues in advance, allowing them to take proactive measures.

In this example, the business problem involves predicting failures due to component malfunctions, answering the question, "What is the probability that a machine will fail in the near future due to a specific component failure?" This is framed as a multi-class classification problem, where a machine learning algorithm is employed to create a predictive model based on historical machine data.

The following sections will guide you through the implementation steps of such a model, including feature engineering, label construction, training, and evaluation. In the next section, we begin by explaining the data sources.

## Data Sources

Common data sources for predictive maintenance problems are:

- **Failure history**: The failure history of a machine or component within the machine.
- **Maintenance history**: The repair history of a machine, e.g. error codes, previous maintenance activities or component replacements.
- **Machine conditions and usage**: The operating conditions of a machine e.g. data collected from sensors.
- **Machine features**: The features of a machine, e.g. engine size, make and model, location.
- **Operator features**: The features of the operator, e.g. gender, past experience.

The data for this example comes from 4 different sources: real-time telemetry data collected from machines, error messages, historical maintenance records that include failures, and machine information such as type and age.



#### Press ▶ to load the data

In [ ]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

telemetry = pd.read_csv('./Data/PdM_telemetry.csv')
errors = pd.read_csv('./Data/PdM_errors.csv')
maint = pd.read_csv('./Data/PdM_maint.csv')
failures = pd.read_csv('./Data/PdM_failures.csv')
machines = pd.read_csv('./Data/PdM_machines.csv')

### Telemetry

The first data source is the telemetry time-series data which consists of voltage, rotation, pressure, and vibration measurements collected from 100 machines in real time averaged over every hour collected during the year 2015. Below, we display the first 5 records in the dataset. A summary of the whole dataset is also provided.

#### Press ▶ to display the telemetry data and its description.

In [ ]:
# format datetime field which comes in as string
telemetry['datetime'] = pd.to_datetime(telemetry['datetime'], format="%Y-%m-%d %H:%M:%S")

print("\033[1m Total number of telemetry records: %d \033[0m" % len(telemetry.index))
display(telemetry.head())
telemetry.describe()

As an example, below is a plot of voltage values for machine ID 1 for the first two months of 2015.

#### Press ▶ to display the voltage values.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plot_df = telemetry.loc[(telemetry['machineID'] == 1) &
                        (telemetry['datetime'] > pd.to_datetime('2015-01-01')) &
                        (telemetry['datetime'] < pd.to_datetime('2015-02-01')), ['datetime', 'volt']]

sns.set_style("darkgrid")
plt.figure(figsize=(12, 6))
plt.plot(plot_df['datetime'], plot_df['volt'])
plt.ylabel('voltage')

# make x-axis ticks legible
adf = plt.gca().get_xaxis().get_major_formatter()
adf.scaled[1.0] = '%m-%d'
plt.xlabel('Date');

### Errors

The second major data source is the error logs. These are non-breaking errors thrown while the machine is still operational and do not constitute as failures. The error date and times are rounded to the closest hour since the telemetry data is collected at an hourly rate.

#### Press ▶ to show the error data and its count records per error ID.

In [ ]:
# format datetime field which comes in as string
errors['datetime'] = pd.to_datetime(errors['datetime'], format="%Y-%m-%d %H:%M:%S")
errors['errorID'] = errors['errorID'].astype('category')

sns.set_style("darkgrid")
plt.figure(figsize=(8, 4))
errors['errorID'].value_counts().plot(kind='bar')
plt.ylabel('Count');

print("\033[1m Total number of error records: %d \033[0m" % len(errors.index))
errors.head()

### Maintenance

These are the scheduled and unscheduled maintenance records which correspond to both regular inspection of components as well as failures. A record is generated if a component is replaced during the scheduled inspection or replaced due to a breakdown. The records that are created due to breakdowns will be called failures which is explained in the later sections. Maintenance data has both 2014 and 2015 records.

#### Press ▶ to display the maintenance data and its count records per component.

In [ ]:
# format datetime field which comes in as string
maint['datetime'] = pd.to_datetime(maint['datetime'], format="%Y-%m-%d %H:%M:%S")
maint['comp'] = maint['comp'].astype('category')

sns.set_style("darkgrid")
plt.figure(figsize=(8, 4))
maint['comp'].value_counts().plot(kind='bar')
plt.ylabel('Count');

print("\033[1m Total number of maintenance records: %d \033[0m" % len(maint.index))
maint.head()

### Machines

This data set includes some information about the machines: model type and age (years in service).

#### Press ▶ to display a sample of the machine data and its count over the years.

In [ ]:
machines['model'] = machines['model'].astype('category')

sns.set_style("darkgrid")
plt.figure(figsize=(8, 6))
_, bins, _ = plt.hist([machines.loc[machines['model'] == 'model1', 'age'],
                       machines.loc[machines['model'] == 'model2', 'age'],
                       machines.loc[machines['model'] == 'model3', 'age'],
                       machines.loc[machines['model'] == 'model4', 'age']],
                       20, stacked=True, label=['model1', 'model2', 'model3', 'model4'])
plt.xlabel('Age (yrs)')
plt.ylabel('Count')
plt.legend();

print("\033[1m Total number of machines: %d \033[0m" % len(machines.index))
machines.head()

### Failures

These are the records of component replacements due to failures. Each record has a date and time, machine ID, and failed component type.

Below is the histogram of the failures due to each component. We see that component 2 causes the most failures.

#### Press ▶ to display a sample of the failure data and its count per component.

In [ ]:
# format datetime field which comes in as string
failures['datetime'] = pd.to_datetime(failures['datetime'], format="%Y-%m-%d %H:%M:%S")
failures['failure'] = failures['failure'].astype('category')

sns.set_style("darkgrid")
plt.figure(figsize=(8, 4))
failures['failure'].value_counts().plot(kind='bar')
plt.ylabel('Count');

print("\033[1m Total number of failures: %d \033[0m" % len(failures.index))
failures.head()

## Feature Engineering

The first step in predictive maintenance applications is feature engineering which requires bringing the different data sources together to create features that best describe a machines's health condition at a given point in time. In the next sections, several feature engineering methods are used to create features based on the properties of each data source.

### Lag Features from Telemetry

Telemetry data almost always comes with time-stamps which makes it suitable for calculating lagging features. A common method is to pick a window size for the lag features to be created and compute rolling aggregate measures such as mean, standard deviation, minimum, maximum, etc. to represent the short term history of the telemetry over the lag window. In the following, rolling mean and standard deviation of the telemetry data over the last 3 hour lag window is calculated for every 3 hours.

#### Press ▶ to show the 3 hour lag features.

In [ ]:
from IPython.display import clear_output

print("Loading data, please wait...")

# Calculate mean values for telemetry features
temp = []
fields = ['volt', 'rotate', 'pressure', 'vibration']
for col in fields:
    temp.append(pd.pivot_table(telemetry,
                               index='datetime',
                               columns='machineID',
                               values=col).resample('3H', closed='left', label='right').mean().unstack())
telemetry_mean_3h = pd.concat(temp, axis=1)
telemetry_mean_3h.columns = [i + 'mean_3h' for i in fields]
telemetry_mean_3h.reset_index(inplace=True)

# repeat for standard deviation
temp = []
for col in fields:
    temp.append(pd.pivot_table(telemetry,
                               index='datetime',
                               columns='machineID',
                               values=col).resample('3H', closed='left', label='right').std().unstack())
telemetry_sd_3h = pd.concat(temp, axis=1)
telemetry_sd_3h.columns = [i + 'sd_3h' for i in fields]
telemetry_sd_3h.reset_index(inplace=True)

clear_output(wait=True)

telemetry_mean_3h.head()

For capturing a longer term effect, 24 hour lag features are also calculated as below.

#### Press ▶ to show the 24 hour lag features.

In [ ]:
print("Loading data, please wait...")
temp = []
fields = ['volt', 'rotate', 'pressure', 'vibration']
for col in fields:
    rolling_mean = pd.pivot_table(telemetry,
                                  index='datetime',
                                  columns='machineID',
                                  values=col).rolling(window=24).mean()    
    resampled = rolling_mean.resample('3H', closed='left', label='right').first().unstack()
    temp.append(resampled)
    
telemetry_mean_24h = pd.concat(temp, axis=1)
telemetry_mean_24h.columns = [i + 'mean_24h' for i in fields]
telemetry_mean_24h.reset_index(inplace=True)
telemetry_mean_24h = telemetry_mean_24h.loc[-telemetry_mean_24h['voltmean_24h'].isnull()]

# repeat for standard deviation
temp = []
fields = ['volt', 'rotate', 'pressure', 'vibration']
for col in fields:
    rolling_std = pd.pivot_table(telemetry,
                                  index='datetime',
                                  columns='machineID',
                                  values=col).rolling(window=24).std()
    resampled = rolling_std.resample('3H', closed='left', label='right').first().unstack()
    temp.append(resampled)
    
telemetry_sd_24h = pd.concat(temp, axis=1)
telemetry_sd_24h.columns = [i + 'sd_24h' for i in fields]
telemetry_sd_24h.reset_index(inplace=True)
telemetry_sd_24h = telemetry_sd_24h.loc[-telemetry_sd_24h['voltsd_24h'].isnull()]

clear_output(wait=True)

# Notice that a 24h rolling average is not available at the earliest timepoints
telemetry_mean_24h.head(10)

Next, the columns of the feature datasets created earlier are merged to create the final feature set from telemetry.

#### Press ▶ to display the final feature set from telemetry.

In [ ]:
# merge columns of feature sets created earlier
telemetry_feat = pd.concat([telemetry_mean_3h,
                            telemetry_sd_3h.iloc[:, 2:6],
                            telemetry_mean_24h.iloc[:, 2:6],
                            telemetry_sd_24h.iloc[:, 2:6]], axis=1).dropna()

print('\033[1mThis is a sample of the data.\033[0m\n')
display(telemetry_feat.head())
print('\n\033[1mThis is the summary of the entire data\033[0m\n')
telemetry_feat.describe()

### Lag Features from Errors

Like telemetry data, errors come with timestamps. An important difference is that the error IDs are categorical values and should not be averaged over time intervals like the telemetry measurements. Instead, we count the number of errors of each type in a lagging window. We begin by reformatting the error data to have one entry per machine per time at which at least one error occurred:

#### Press ▶ to display the updated error data.

In [ ]:
# create a column for each error type
error_count = pd.get_dummies(errors.set_index('datetime')).reset_index()
error_count.columns = ['datetime', 'machineID', 'error1', 'error2', 'error3', 'error4', 'error5']

# combine errors for a given machine in a given hour
error_count = error_count.groupby(['machineID', 'datetime']).sum().reset_index()
error_count.head(13)

Now we add blank entries for all other hourly timepoints (since no errors occurred at those times):

#### Press ▶ to display the summary of the updated error data.

In [ ]:
error_count = telemetry[['datetime', 'machineID']].merge(error_count, on=['machineID', 'datetime'], how='left').fillna(0.0)
error_count.describe()

Finally, we can compute the total number of errors of each type over the last 24 hours, for timepoints taken every three hours:

#### Press ▶ to show a sample of the error data and its summary.

In [ ]:
print("Loading data, please wait...")
temp = []
fields = ['error%d' % i for i in range(1,6)]
for col in fields:
    rolling_sum = pd.pivot_table(error_count,
                                  index='datetime',
                                  columns='machineID',
                                  values=col).rolling(window=24).sum()    
    resampled = rolling_sum.resample('3H', closed='left', label='right').first().unstack()
    temp.append(resampled)
    
error_count = pd.concat(temp, axis=1)
error_count.columns = [i + 'count' for i in fields]
error_count.reset_index(inplace=True)
error_count = error_count.dropna()

clear_output(wait=True)

print('\033[1mThis shows a sample of the data.\033[0m\n')
display(error_count.head())

print('\n\033[1mThis shows the summary of the data.\033[0m\n')
error_count.describe()

### Days Since Last Replacement from Maintenance

A crucial data set in this example is the maintenance records which contain the information of component replacement records. Possible features from this data set can be, for example, the number of replacements of each component in the last 3 months to incorporate the frequency of replacements. However, more relevent information would be to calculate how long it has been since a component is last replaced as that would be expected to correlate better with component failures since the longer a component is used, the more degradation should be expected. 

As a side note, creating lagging features from maintenance data is not as straightforward as for telemetry and errors, so the features from this data are generated in a more custom way. This type of ad-hoc feature engineering is very common in predictive maintenance since domain knowledge plays a big role in understanding the predictors of a problem. In the following, the days since last component replacement are calculated for each component type as features from the maintenance data. 

#### Press ▶ to show a sample of the maintenance data and its summary.

In [ ]:
import numpy as np
print("Loading data, please wait...")
# create a column for each error type
comp_rep = pd.get_dummies(maint.set_index('datetime')).reset_index()
comp_rep.columns = ['datetime', 'machineID', 'comp1', 'comp2', 'comp3', 'comp4']

# combine repairs for a given machine in a given hour
comp_rep = comp_rep.groupby(['machineID', 'datetime']).sum().reset_index()

# add timepoints where no components were replaced
comp_rep = telemetry[['datetime', 'machineID']].merge(comp_rep,
                                                      on=['datetime', 'machineID'],
                                                      how='outer').fillna(0).sort_values(by=['machineID', 'datetime'])

components = ['comp1', 'comp2', 'comp3', 'comp4']
for comp in components:
    # convert indicator to most recent date of component change
    comp_rep.loc[comp_rep[comp] < 1, comp] = None
    comp_rep.loc[-comp_rep[comp].isnull(), comp] = comp_rep.loc[-comp_rep[comp].isnull(), 'datetime']
    
    # forward-fill the most-recent date of component change
    comp_rep[comp] = comp_rep[comp].fillna(method='ffill')

# remove dates in 2014 (may have NaN or future component change dates)    
comp_rep = comp_rep.loc[comp_rep['datetime'] > pd.to_datetime('2015-01-01')]

# replace dates of most recent component change with days since most recent component change
for comp in components:
    comp_rep[comp] = (comp_rep['datetime'] - comp_rep[comp]) / np.timedelta64(1, 'D')
    
clear_output(wait=True)

print('\033[1mThis is sample of the data.\033[0m\n')
display(comp_rep.head())

print('\n\033[1mThis is the summary of the data.\033[0m\n')
comp_rep.describe()

### Machine Features

The machine features can be used without further modification. These include descriptive information about the type of each machine and its age (number of years in service). If the age information had been recorded as a "first use date" for each machine, a transformation would have been necessary to turn those into a numeric values indicating the years in service.

Lastly, we merge all the feature data sets we created earlier to get the final feature matrix.

#### Press ▶ to show a sample of the final feature data and its summary.

In [ ]:
final_feat = telemetry_feat.merge(error_count, on=['datetime', 'machineID'], how='left')
final_feat = final_feat.merge(comp_rep, on=['datetime', 'machineID'], how='left')
final_feat = final_feat.merge(machines, on=['machineID'], how='left')

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(final_feat)
#display(final_feat.head())
final_feat.describe()

## Label Construction

When using multi-class classification for predicting failure due to a problem, labelling is done by taking a time window prior to the failure of an asset and labelling the feature records that fall into that window as "about to fail due to a problem" while labelling all other records as "normal." This time window should be picked according to the business case: in some situations it may be enough to predict failures hours in advance, while in others days or weeks may be needed to allow e.g. for arrival of replacement parts.

The prediction problem for this example scenerio is to estimate the probability that a machine will fail in the near future due to a failure of a certain component. More specifically, the goal is to compute the probability that a machine will fail in the next 24 hours due to a certain component failure (component 1, 2, 3, or 4). Below, a categorical `failure` feature is created to serve as the label. All records within a 24 hour window before a failure of component 1 have `failure=comp1`, and so on for components 2, 3, and 4; all records not within 24 hours of a component failure have `failure=none`.

#### Press ▶ to show the feature data and its "failure" label.

In [ ]:
labeled_features = final_feat.merge(failures, on=['datetime', 'machineID'], how='left')
labeled_features = labeled_features.fillna(method='bfill', limit=7) # fill backward up to 24h

for col in labeled_features.select_dtypes(include='category').columns:
    labeled_features[col] = labeled_features[col].astype(str)

labeled_features = labeled_features.replace('nan', 'none')

labeled_features.head()

Below is an example of records that are labeled as `failure=comp4` in the failure column. Notice that the first 8 records all occur in the 24-hour window before the first recorded failure of component 4. The next 8 records are within the 24 hour window before another failure of component 4.

#### Press ▶ to show an example of "comp4" failure.

In [ ]:
labeled_features.loc[labeled_features['failure'] == 'comp4'][:16]

## Modelling

### Training, Validation and Testing

When working with time-stamped data as in this example, record partitioning into training, validation, and test sets should be performed carefully to prevent overestimating the performance of the models. In predictive maintenance, the features are usually generated using lagging aggregates: records in the same time window will likely have identical labels and similar feature values. These correlations can give a model an "unfair advantage" when predicting on a test set record that shares its time window with a training set record. We therefore partition records into training, validation, and test sets in large chunks, to minimize the number of time intervals shared between them.

Predictive models have no advance knowledge of future chronological trends: in practice, such trends are likely to exist and to adversely impact the model's performance. To obtain an accurate assessment of a predictive model's performance, we recommend training on older records and validating/testing using newer records.

For both of these reasons, a time-dependent record splitting strategy is an excellent choice for predictive maintenace models. The split is effected by choosing a point in time based on the desired size of the training and test sets: all records before the timepoint are used for training the model, and all remaining records are used for testing. (If desired, the timeline could be further divided to create validation sets for parameter selection.) To prevent any records in the training set from sharing time windows with the records in the test set, we remove any records at the boundary -- in this case, by ignoring 24 hours' worth of data prior to the timepoint.

#### Press ▶ to train the model and predict the test results.

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

# make test and training splits
threshold_dates = [[pd.to_datetime('2015-07-31 01:00:00'), pd.to_datetime('2015-08-01 01:00:00')],
                   [pd.to_datetime('2015-08-31 01:00:00'), pd.to_datetime('2015-09-01 01:00:00')],
                   [pd.to_datetime('2015-09-30 01:00:00'), pd.to_datetime('2015-10-01 01:00:00')]]

test_results = []
models = []
for last_train_date, first_test_date in threshold_dates:
    # split out training and test data
    train_y = labeled_features.loc[labeled_features['datetime'] < last_train_date, 'failure']
    train_X = pd.get_dummies(labeled_features.loc[labeled_features['datetime'] < last_train_date].drop(columns=['datetime',
                                                                                                        'machineID',
                                                                                                        'failure']))
    test_X = pd.get_dummies(labeled_features.loc[labeled_features['datetime'] > first_test_date].drop(columns=['datetime',
                                                                                                       'machineID',
                                                                                                       'failure']))
    # train and predict using the model, storing results for later
    my_model = HistGradientBoostingClassifier(max_depth = 9, verbose=2, random_state=42)
    my_model.fit(train_X, train_y)
    test_result = pd.DataFrame(labeled_features.loc[labeled_features['datetime'] > first_test_date])
    test_result['predicted_failure'] = my_model.predict(test_X)
    test_results.append(test_result)
    models.append(my_model)

## Evaluation

In predictive maintenance, machine failures are usually rare occurrences in the lifetime of the assets compared to normal operations. This causes an imbalance in the label distribution, which usually causes poor performance as algorithms tend to classify majority class examples better at the expense of minority class examples, as the total misclassification error is much improved when the majority class is labeled correctly.  This causes low recall rates, although accuracy can be high, and becomes a larger problem when the cost of false alarms to the business is very high. To help with this problem, sampling techniques such as oversampling of the minority examples are usually used along with more sophisticated techniques that are not covered in this notebook.

#### Data Imbalance

Visualize the categories distribution. We clearly see that the "none" class is dominant and there is a data imbalance.

#### Press ▶ to display the distribution of the target classes.

In [ ]:
sns.set_style("darkgrid")
plt.figure(figsize=(8, 4))
labeled_features['failure'].value_counts().plot(kind='bar')
plt.xlabel('Component failing')
plt.ylabel('Count');

#### Baseline Classification Metrics

Also, due to the class imbalance problem, it is important to look at evaluation metrics other than accuracy alone and compare those metrics to the baseline metrics, which are computed when random chance is used to make predictions rather than a machine learning model.  The comparison will better highlight the value and benefits of using a machine learning model.

In the following, we use an evaluation function that computes many important evaluation metrics and baseline metrics for classification problems.

#### Press ▶ to display the confusion matrices the three different splits and the evaluation results.

In [ ]:
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score
import itertools

def Evaluate(predicted, actual, labels):
    output_labels = []
    output = []
    
    # Calculate and display confusion matrix
    cm = confusion_matrix(actual, predicted, labels=labels)

    # Plotting the confusion matrix
    plt.figure(figsize=(5, 4))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.xlabel('True Labels')
    plt.ylabel('Predicted Labels')

    plt.grid(False)

    # Annotating the confusion matrix
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.show()
    
    #print('Confusion matrix\n- x-axis is true labels (none, comp1, etc.)\n- y-axis is predicted labels')
    #print(cm)
    
    # Calculate precision, recall, and F1 score
    accuracy = np.array([float(np.trace(cm)) / np.sum(cm)] * len(labels))
    precision = precision_score(actual, predicted, average=None, labels=labels)
    recall = recall_score(actual, predicted, average=None, labels=labels)
    f1 = 2 * precision * recall / (precision + recall)
    output.extend([accuracy.tolist(), precision.tolist(), recall.tolist(), f1.tolist()])
    output_labels.extend(['accuracy', 'precision', 'recall', 'F1'])
    
    # Calculate the macro versions of these metrics
    output.extend([[np.mean(precision)] * len(labels),
                   [np.mean(recall)] * len(labels),
                   [np.mean(f1)] * len(labels)])
    output_labels.extend(['macro precision', 'macro recall', 'macro F1'])
    
    # Find the one-vs.-all confusion matrix
    cm_row_sums = cm.sum(axis = 1)
    cm_col_sums = cm.sum(axis = 0)
    s = np.zeros((2, 2))
    for i in range(len(labels)):
        v = np.array([[cm[i, i],
                       cm_row_sums[i] - cm[i, i]],
                      [cm_col_sums[i] - cm[i, i],
                       np.sum(cm) + cm[i, i] - (cm_row_sums[i] + cm_col_sums[i])]])
        s += v
    s_row_sums = s.sum(axis = 1)
    
    # Add average accuracy and micro-averaged  precision/recall/F1
    avg_accuracy = [np.trace(s) / np.sum(s)] * len(labels)
    micro_prf = [float(s[0,0]) / s_row_sums[0]] * len(labels)
    output.extend([avg_accuracy, micro_prf])
    output_labels.extend(['average accuracy',
                          'micro-averaged precision/recall/F1'])
    
    # Compute metrics for the majority classifier
    mc_index = np.where(cm_row_sums == np.max(cm_row_sums))[0][0]
    cm_row_dist = cm_row_sums / float(np.sum(cm))
    mc_accuracy = 0 * cm_row_dist; mc_accuracy[mc_index] = cm_row_dist[mc_index]
    mc_recall = 0 * cm_row_dist; mc_recall[mc_index] = 1
    mc_precision = 0 * cm_row_dist
    mc_precision[mc_index] = cm_row_dist[mc_index]
    mc_F1 = 0 * cm_row_dist;
    mc_F1[mc_index] = 2 * mc_precision[mc_index] / (mc_precision[mc_index] + 1)
    output.extend([mc_accuracy.tolist(), mc_recall.tolist(),
                   mc_precision.tolist(), mc_F1.tolist()])
    output_labels.extend(['majority class accuracy', 'majority class recall',
                          'majority class precision', 'majority class F1'])
        
    # Random accuracy and kappa
    cm_col_dist = cm_col_sums / float(np.sum(cm))
    exp_accuracy = np.array([np.sum(cm_row_dist * cm_col_dist)] * len(labels))
    kappa = (accuracy - exp_accuracy) / (1 - exp_accuracy)
    output.extend([exp_accuracy.tolist(), kappa.tolist()])
    output_labels.extend(['expected accuracy', 'kappa'])
    

    # Random guess
    rg_accuracy = np.ones(len(labels)) / float(len(labels))
    rg_precision = cm_row_dist
    rg_recall = np.ones(len(labels)) / float(len(labels))
    rg_F1 = 2 * cm_row_dist / (len(labels) * cm_row_dist + 1)
    output.extend([rg_accuracy.tolist(), rg_precision.tolist(),
                   rg_recall.tolist(), rg_F1.tolist()])
    output_labels.extend(['random guess accuracy', 'random guess precision',
                          'random guess recall', 'random guess F1'])
    
    # Random weighted guess
    rwg_accuracy = np.ones(len(labels)) * sum(cm_row_dist**2)
    rwg_precision = cm_row_dist
    rwg_recall = cm_row_dist
    rwg_F1 = cm_row_dist
    output.extend([rwg_accuracy.tolist(), rwg_precision.tolist(),
                   rwg_recall.tolist(), rwg_F1.tolist()])
    output_labels.extend(['random weighted guess accuracy',
                          'random weighted guess precision',
                          'random weighted guess recall',
                          'random weighted guess F1'])

    output_df = pd.DataFrame(output, columns=labels)
    output_df.index = output_labels
                  
    return output_df

evaluation_results = []
for i, test_result in enumerate(test_results):
    print('\n\033[1mSplit %d:\033[0m' % (i+1))
    evaluation_result = Evaluate(actual = test_result['failure'],
                                 predicted = test_result['predicted_failure'],
                                 labels = ['none', 'comp1', 'comp2', 'comp3', 'comp4'])
    evaluation_results.append(evaluation_result)
evaluation_results[0]  # show full results for first split only

#### Evaluation Highlights
In predictive maintenance, we are often most concerned with how many of the actual failures were predicted by the model, i.e. the model's recall. (Recall becomes more important as the consequences of *false negatives* -- true failures that the model did not predict -- exceed the consequences of *false positives*, viz. false prediction of impending failure.) Below, we compare the recall rates for each failure type for the three models. The recall rates for all components as well as no failure are all above 90% meaning the model was able to capture above 90% of the failures correctly.

#### Press ▶ to show the recall values per split per class.

In [ ]:
recall_df = pd.DataFrame([evaluation_results[0].loc['recall'].values,
                          evaluation_results[1].loc['recall'].values,
                          evaluation_results[2].loc['recall'].values],
                         columns = ['none', 'comp1', 'comp2', 'comp3', 'comp4'],
                         index = ['recall for first split',
                                  'recall for second split',
                                  'recall for third split'])
recall_df

## Summary

This notebook outlines the process of implementing a predictive maintenance model using a sample scenario where machine failures occur due to specific component malfunctions. The objective is to predict these failures. The example datasets illustrate key steps in predictive maintenance, including feature engineering, labeling, training, and evaluation.